# SPFlow 1.0.0 Design Concept

Design basis:
* Easy-to use and extend
<br>

* Functional design (using dispatch)
<br>

* Modular building blocks that can be stacked and nested
    * Allows for quick design of large models
    * Allows to combine existing or extend existing models
<br>


* Support multiple back-end with one-to-one mappings between backends
    * Allows for easy model sharing and conversion to favorite back-end
    * Base back-end should use explicit node modules as lowest basic blocks
        * E.g allows alternative node-wise evaluations (e.g. for computing p-values)
    * Other back-ends may use optimized modules (i.e. implicit nodes)
        * Still need to be mappable to base-backend

![overview](uml/all.svg)

##### Imports

In [1]:
from typing import Any, List, Tuple, Set, Dict, Union, Optional
from abc import ABC, abstractmethod, abstractproperty
from dataclasses import dataclass
import itertools
from multipledispatch import dispatch

import numpy as np
from scipy.stats import norm

## NetworkType Class

* All network types (e.g. `SPN`, `BN`, ...) inherit from it
* Network types do not need to actually implement anything
* Simply needed to dispatch on network type objects (e.g. for computing scopes, likelihoods, sampling etc.)

![network_type](uml/network_type.svg)

In [2]:
class NetworkType(ABC):
    """Abstract base class for network types."""
    pass

#### SPN Network Type

In [3]:
class SPN(NetworkType):
    """Sum-Product Network (SPN) network type."""
    pass

## Scope Class

Attributes:
* `scope`: set of indices (scope)
* `ntype`: network type this scope is in reference to

The class can, for example, overwride `__add__` operator to merge scopes based on their network types.

<b>TODO:</b>
* Network types and modules may have to check whether or not this merging is valid (e.g. sum- vs product-nodes)
* Conditional/evidence variables


![scope](uml/scope.svg)

In [4]:
class Scope:
    """Class representing Variable scopes."""
    def __init__(self, ntype: NetworkType, variables: Set[int]) -> None:
        self.variables = set(variables)
        self.network_type = ntype

    def __repr__(self) -> str:
        return f"Scope({self.variables})"
    
    def __add__(self, other: "Scope") -> "Scope":
        "Dispatches scope merging based on network types."
        return merge_scope(self, other)
    
    def __len__(self) -> int:
        return len(self.variables)

Merging scopes is automatically dispatched to merging scopes based on their respective network type contexts.

In [5]:
@dispatch(Scope, Scope)
def merge_scope(scope1: Scope, scope2: Scope) -> Scope:
    """Generic intermediate function to dispatch merging based on network types."""
    return merge_scope(scope1.network_type, scope1, scope2.network_type, scope2)

To register a new merge operation (e.g. between two `SPN` scopes): dispatch appropriate function.

In [6]:
@dispatch(SPN, Scope, SPN, Scope)
def merge_scope(ntype1: SPN, scope1: Scope, ntype2: SPN, scope2: Scope) -> Scope:
    """Merges two SPN scopes."""
    return Scope(SPN(), set.union(scope1.variables, scope2.variables))

##### Examples

In [7]:
s1 = Scope(SPN(), [0,1])
print(f"Scope 1: {s1.variables}")

Scope 1: {0, 1}


In [8]:
s2 = Scope(SPN(), [1,2,3])
print(f"Scope 2: {s2.variables}")

Scope 2: {1, 2, 3}


In [9]:
s = s1+s2
print(f"Merged scope: {s.variables}")

Merged scope: {0, 1, 2, 3}


### Scope Array

Array class inheriting from `np.ndarray`. Alterantively one can just pass `dtype=Scope` as an argument to `np.ndarray`.

Can store scopes with different scope length in an array format that can easily be propagated through a model (e.g. similar to likelihood values)

In [10]:
class ScopeArray(np.ndarray):
    """Numpy array with Scope object elements."""
    def __new__(cls, data) -> np.ndarray:
        return np.array(data, dtype=Scope)

##### Examples

In [11]:
ScopeArray([s1, s2])

array([Scope({0, 1}), Scope({1, 2, 3})], dtype=object)

Also allows to use appropriate numpy-operations

In [12]:
ScopeArray([s1, s2]).sum()

Scope({0, 1, 2, 3})

## Module Class

Every module inherits from this class.

Each module must also implement `__len__(self)` to return the number of (implicit or explicit) output node of this module.

![module](uml/module.svg)

In [13]:
class Module(ABC):
    """Abstract base class for modules.
    
    Args:
        children: list of child modules (may be empty for terminal modules).
    """
    def __init__(self, children: List["Module"]=[]) -> None:
        # set child modules
        self.children = children
        
        # infer number of inputs from children (and their numbers of outputs)
        child_num_outputs = [child.n_out for child in self.children]
        child_cum_outputs = np.cumsum(child_num_outputs)
        
        self.n_in = sum(child_num_outputs, 0)

        # compute conversion from input ids corresponding child and output id (Saves computation at run-time)
        self.input_to_output_id_dict = {}
        
        for input_id in range(self.n_in):
            # get child module for corresponding input
            child_id = np.sum(child_cum_outputs <= input_id, axis=0).tolist()
            # get output id of child module for corresponding input
            output_id = input_id-(child_cum_outputs[child_id]-child_num_outputs[child_id])
            
            self.input_to_output_id_dict[input_id] = (child_id, output_id)

    @abstractproperty
    def n_out(self) -> int:
        """Specifies the number of outputs, i.e. (implicit of explicit) output nodes."""
        pass

    def input_to_output_id(self, input_id) -> Tuple[int, int]:
        """Helper method to convert an input id to a corresponding child and child output id."""
        return self.input_to_output_id_dict[input_id]

## Sampling

Sampling should return an array in a form that could immediately be used to infer likelihoods of that same module. That entails, that the RVs are in ascending order and that no RVs are skipped (e.g. sampling over a scope of `[0,2,5]` should return an array of size `(n,6)` to accommodate for all scope RVs at their respective indices while leaving all other entries as NaNs; `n` is the number of samples).

API:
- `sample(module)` should return return a single sample (size `(1,m)`)
- `sample(module, n)` should return return `n` samples (size `(n,m)`)
- `sample(module, array)` should fill the specified array in-place and return it as well (array must be of appropriate size). This also allows to specify incomplete data that is not replaced during sampled and whose likelihoods are taken into account while sampling.|

When sampling for multiple instances, we need to keep track which modules are supposed to sample which instances.

Example:
![sampling_context_example](img/sampling_context_example.drawio.svg)
In this case, the sum node samples a branch (B or C) for each instance, so A and B are sampling into the same data array, but get different instances to sample.

In the multi-output case (i.e. modules, see below) one additionally needs to specify which outputs are supposed to be sampled.

Example:
![sampling_context_example](img/sampling_context_example_2.drawio.svg)
Here, we essentially have the same graph as above, but `sample(...)` is called on the same child module for all instances.

`SamplingContext` class to track which instances to sample and which module outputs to sample from. Keeps a list of instance ids to fill with samples and corresponding output ids to sample from for these instances for a given module.

![sampling_context](uml/sampling_context.svg)

In [14]:
class SamplingContext:
    """Keeps track of instance ids to sample and which output ids to sample from (relevant for modules with multiple outputs).
    
    Args:
        instance_ids: list of ints representing the instances to sample.
        output_ids: list of lists of ints representing the output ids for the corresponding instances to sample from (relevant for multi-output module).
                    As a shorthand, '[]' implies to sample from all outputs for a given instance id.
    """
    def __init__(self, instance_ids: List[int], output_ids: Optional[List[List[int]]]=None) -> None:

        if output_ids is None:
            # assume sampling over all outputs (i.e. [])
            output_ids = [[] for _ in instance_ids]
        
        if (len(output_ids) != len(instance_ids)):
            raise ValueError(f"Number of specified instance ids {len(instance_ids)} does not match specified number of output ids {len(output_ids)}.")

        self.instance_ids = instance_ids
        self.output_ids = output_ids

    def select(self, ids: List[int]) -> "SamplingConctext":
        """Selects a subset of the instance ids and their corresponding output ids in a new sampling context object."""
        selection = [pair for i, pair in enumerate(zip(self.instance_ids, self.output_ids)) if i in ids]
        return SamplingContext(*tuple(list(item) for item in zip(*selection)))

    def append(self, instance_ids: List[int], output_ids: List[List[int]]) -> None:
        """Adds new instance ids and corresponding output ids to the sampling context."""
        if(len(instance_ids) != len(output_ids)):
            raise ValueError(f"Number of specified instance ids {len(instance_ids)} does not match specified number of output ids {len(output_ids)}.")
        
        self.instance_ids += instance_ids
        self.output_ids += output_ids

    def remove(self, ids: List[int]) -> None:
        """Removes a subset of the instance ids and their corresponding output ids from the sampling context object."""
        selection = [pair for i, pair in enumerate(zip(self.instance_ids, self.output_ids)) if i not in ids]
        self.instance_ids, self.output_ids = tuple(list(item) for item in zip(*selection))
    
    def is_valid(self, data: Optional[np.ndarray]=None, module: Optional[Module]=None) -> bool:
        """Returns a boolean whether or not the sampling context object is valid. Additionally, a data array and a module may be specified)."""
        
        # check if there are any duplicate instance ids
        if len(set(self.instance_ids)) != len(self.instance_ids):
            return False
        
        # validate individual instance ids
        for instance_id in self.instance_ids:
            # check if all instance ids are greater or equal to 0
            if instance_id < 0:
                return False
            # if a data array is specified, then also check if the instance ids are valid for it
            if (data is not None) and (instance_id >= data.shape[0]):
                return False
        
        # check if number of output ids matches number of input ids
        if len(self.instance_ids) != len(self.output_ids):
            return False
        
        # check if any output id lists contain duplicates
        for out_ids in self.output_ids:
            if len(set(out_ids)) != len(out_ids):
                return False
        
        # validate individual output ids
        for out_ids in self.output_ids:
            for out_id in out_ids:
                # Check if all output ids are greater of equal to 0
                if out_id < 0:
                    return False
                # if a module is specified, then also check if the output ids are valid indices for it
                if (module is not None) and (out_id >= module.n_out):
                    return False
        
        return True

##### Examples

In [15]:
sc = SamplingContext([0,2,4,6],[[1],[],[],[4]])
print(sc.instance_ids, sc.output_ids)

[0, 2, 4, 6] [[1], [], [], [4]]


In [16]:
# select certain instance ids (e.g. to pass to a child module for sampling)
sc2 = sc.select([1,3])
print(sc2.instance_ids, sc2.output_ids)

[2, 6] [[], [4]]


In [17]:
# remove certain instance ids
sc.remove([1,3])
print(sc.instance_ids, sc.output_ids)

[0, 4] [[1], []]


Dispatch sampling for all modules without specified data tensor (see API-notes above)

In [18]:
@dispatch(Module, ll_cache=dict)
def sample(module: Module, ll_cache: dict={}) -> np.ndarray:
    """Dispatches sampling a single instance from the module."""
    return sample(module, 1, ll_cache=ll_cache)

@dispatch(Module, int, ll_cache=dict, sampling_context=SamplingContext)
def sample(module: Module, n_samples: int, ll_cache: dict={}, sampling_context: SamplingContext=None) -> np.ndarray:
    """Creates an array for n samples and dispatches sampling to fill the array"""
    if(not sampling_context):
        # create sampling context (assume all instances are sampled from)
        sampling_context = SamplingContext(list(range(n_samples)), [[] for _ in range(n_samples)])

    # get module scope and largest random variable id
    module_scope = scope(module).squeeze().tolist()
    max_rv = max(module_scope.variables)

    # create appropriate data tensor to fill
    data = np.full((n_samples,max_rv+1), float("nan"))

    # sample and fill data tensor
    return sample(module, data, ll_cache=ll_cache, sampling_context=sampling_context)

## (Log-)Likelihood

In general, we would like to offer both `likelihood` and `loglikelihood` routines. For better comprehension, we only implement `likelihood` in this module. Note, that we can easily implement one from the other by simply calling `log(..)` or `exp(..)`, respectively.

API:
- `likelihood(module, data)`: should return an array of the size `(n,1)`. `np.nan` values are marginalized over (not necessarily demonstrated here).
- `likelihood(module, data, dict)`: specifies a dictionary (empty, partially or fully filled). For each module check if it is in the dictionary keys and reuses the stored value or computes it and updates the cache afterwards. The cache can then be reused for later calls (with the same data) or e.g. for the `sampling` function.

### Memoization Decorator

E.g. can wrap the `likelihood` function and automatically check the cache and re-use or fill it.

**NOTE**: the `memoize` decorate must come after the `dispatch` decorator (before in the decorator evaluation logic), so that the memoized version of the function is being dispatched!

![memoize](uml/memoize.svg)

In [19]:
from functools import wraps

def memoize(f):
    """Wraps a function to automatically check against a cache ('cache' keyword argument) using the first argument as the key.
    If present, the cached value is returned, otherwise it is computed using the wrapped function stored in the cache.
    """
    @wraps(f)
    def memoized_f(*args, **kwargs):
        # get cache (initialize if non specified)
        kwargs.setdefault("cache", {})
        cache = kwargs["cache"]

        # args contains key variable to be used for cache (assumes key variable is first positional argument to f)
        if len(args) > 0:
            key = args[0]
        # key variable must be part of kwargs
        else:
            raise ValueError("No argument to cache against")

        if key not in cache:
            # compute result and update cache
            cache[key] = f(*args, **kwargs)

        return cache[key]
    return memoized_f

## Node Modules

Basic `LeafNode`,`SumNode` and `ProductNode` classes.

Every class dispatches `likelihood(...)`, `scope(...)` and `sample(...)`.

![nodes](uml/nodes.svg)

### Abstract Node Module

Fixed outputs size (i.e. `__len__`) of 1

In [20]:
class Node(Module, ABC):
    """Represents basic nodes, i.e. the smallest building block for non-optimized networks."""
    @property
    def n_out(self) -> int:
        return 1

### Leaf Nodes

**Note**: `LeafNode` should actually be an abstract base class for leaf nodes to inherit from. In this example we simply implement `LeafNode` as a one-dimensional Gaussian distribution for demonstration purposes.

In [21]:
class LeafNode(Node):
    """Basic univariate leaf node. Here, implemented as a Gaussian.

    Args:
        scope: scope of the distribution.
        mean: mean of the distribution.
        std: standard deviation of the distribution
    """
    def __init__(self, scope: Scope, mean=0.0, std=1.0) -> None:
        
        if(len(scope) != 1):
            raise ValueError("Scope to large for univariate leaf node.")
        
        super(LeafNode, self).__init__()
        
        self.scope = scope
        self.mean = mean
        self.std = std
        self.dist = norm(loc=mean, scale=std)

@dispatch(LeafNode, cache=dict)
@memoize
def scope(module: LeafNode, cache: dict={}) -> ScopeArray:
    """Simply returns the scope of the module, since it's a leaf node."""
    return ScopeArray([[module.scope]])

@dispatch(LeafNode, np.ndarray, cache=dict)
@memoize
def likelihood(module: LeafNode, data: np.ndarray, cache: dict={}) -> np.ndarray:
    """Returns the likelihood for each data instance. Marginalizes (likelihood of 1) for NaN values."""
    likelihoods = np.ones((data.shape[0], len(module.scope)))
    inputs = data[:, list(module.scope.variables)]

    marg_ids = np.isnan(inputs).sum(axis=1) == len(module.scope)
    likelihoods[~marg_ids, :] = module.dist.pdf(inputs[~marg_ids, :])

    return likelihoods

@dispatch(LeafNode, np.ndarray, ll_cache=dict, sampling_context=SamplingContext)
def sample(module: LeafNode, data: np.ndarray, ll_cache: dict={}, sampling_context: Optional[SamplingContext]=None) -> np.ndarray:
    """Samples and fills the indices specified in the sampling context."""

    if sampling_context:
        if not sampling_context.is_valid(data, module):
            # invalid sampling context
            raise ValueError(f"Specified sampling context is invalid for specified data array and module.")
    else:
        # create sampling context (assume all instances and all output nodes are to be used)
        sampling_context = SamplingContext(list(range(data.shape[0])), [[] for _ in range(data.shape[0])]) 

    scope_vars = list(module.scope.variables)

    # sample ids (nan entries)
    sample_ids = (np.isnan(data[:, scope_vars]).sum(axis=1) == len(scope_vars))
    
    # get mask for sampling context
    instance_id_mask = np.zeros(data.shape[0]).astype(bool)
    instance_id_mask[sampling_context.instance_ids] = True

    # filter instance ids according to sampling context
    sample_ids &= instance_id_mask

    # sample
    data[sample_ids, list(module.scope.variables)] = np.random.normal(module.mean, module.std, sample_ids.sum())

    return data

### Product Nodes

In [22]:
class ProductNode(Node):
    """Simple SPN product node. Child nodes (explicit or implicit) are assumed to have pairwise disjoint scopes.
    
    Args:
        children: list of child modules.
    """
    def __init__(self, children: List[Module]) -> None:
        
        if(len(children) == 0):
            raise ValueError(f"List of child modules for ProductNode is empty.")

        super(ProductNode, self).__init__(children)

@dispatch(ProductNode, cache=dict)
@memoize
def scope(module: ProductNode, cache: dict={}) -> ScopeArray:
    """Returns merged child scopes.""" 
    scopes = np.concatenate([scope(child, cache=cache) for child in module.children], axis=1)
    return scopes.sum(keepdims=True)

@dispatch(ProductNode, np.ndarray, cache=dict)
@memoize
def likelihood(module: ProductNode, data: np.ndarray, cache: dict={}) -> np.ndarray:
    """Returns the product of the child likelihoods.""" 
    inputs = np.concatenate([likelihood(child, data, cache=cache) for child in module.children], axis=1)
    return np.prod(inputs, axis=1, keepdims=True)

@dispatch(ProductNode, np.ndarray, ll_cache=dict, sampling_context=SamplingContext)
def sample(module: ProductNode, data: np.ndarray, ll_cache: dict={}, sampling_context=None) -> np.ndarray:
    """Samples from all branches (i.e. nodes/module outputs).""" 
    
    if sampling_context:
        if not sampling_context.is_valid(data, module):
            # invalid sampling context
            raise ValueError(f"Specified sampling context is invalid for specified data array and module.")
    else:
        # create sampling context (assume all instances and all output nodes are to be used)
        sampling_context = SamplingContext(list(range(data.shape[0])), [[] for _ in range(data.shape[0])])

    # sample from all branches (i.e. inputs)
    for child in module.children:
        # create sampling context for child (same instance ids but different output ids now for new module)
        child_sampling_context = SamplingContext(sampling_context.instance_ids, [[] for _ in range(len(sampling_context.instance_ids))])
        # sample from child
        sample(child, data, ll_cache=ll_cache, sampling_context=child_sampling_context)
        

    return data

### Sum Nodes

In [23]:
class SumNode(Node):
    """Simple SPN sum node. merged child scopes. All child nodes (explicit or implicit) are assumed to have the same scopes.
    
    Args:
        children: list of child modules.
    """ 
    def __init__(self, children) -> None:
        
        if(len(children) == 0):
            raise ValueError(f"List of child modules for SumNode is empty.")
        
        super(SumNode, self).__init__(children)
        
        self.weights = np.random.rand(self.n_in)
        self.weights /= self.weights.sum()

@dispatch(SumNode, cache=dict)
def scope(module: SumNode, cache: dict={}) -> ScopeArray:
    """Returns merged child scopes."""
    scopes = np.concatenate([scope(child, cache=cache) for child in module.children])
    return scopes.sum(keepdims=True)

@dispatch(SumNode, np.ndarray, cache=dict)
@memoize
def likelihood(module: SumNode, data: np.ndarray, cache: dict={}) -> np.ndarray:
    """Returns the weighted sum of the child likelihoods."""
    inputs = np.concatenate([likelihood(child, data, cache=cache) for child in module.children], axis=1)
    return (module.weights*inputs).sum(axis=1, keepdims=True)

@dispatch(SumNode, np.ndarray, ll_cache=dict, sampling_context=SamplingContext)
def sample(module: SumNode, data: np.ndarray, ll_cache: dict={}, sampling_context=None) -> np.ndarray:
    """Samples a branch for each instance (taking likelihoods into account)."""
    
    if sampling_context:
        if not sampling_context.is_valid(data, module):
            # invalid sampling context
            raise ValueError(f"Specified sampling context is invalid for specified data array and module.")
    else:
        # create sampling context (assume all instances and all output nodes are to be used)
        sampling_context = SamplingContext(list(range(data.shape[0])), [[] for _ in range(data.shape[0])])

    # get likelihoods for children (using cache; only use relevant instances to save computation)
    child_likelihoods = np.concatenate([likelihood(child, data[sampling_context.instance_ids], cache=ll_cache) for child in module.children], axis=1)
    
    # sample branch for each instance id
    choices = []
    sampling_probs = child_likelihoods * module.weights
    
    for probs in sampling_probs:
        # TODO: normalize? -> yes have to sum to 1
        probs_norm = probs * (1 / np.sum(probs))
        choices.append(np.random.choice(list(range(probs.shape[0])), p=probs_norm))

    choices = np.array(choices)

    # get number of outputs per child module
    child_num_outputs = np.array([child.n_out for child in module.children])
    child_cum_outputs = np.cumsum(child_num_outputs)

    # for each unique sampled branch
    for branch_id in np.unique(choices):
        # group instances by sampled branch
        child_sample_ids = np.where(choices == branch_id)[0]

        # get corresponding child and output id for sampled branch
        child_id, output_id = module.input_to_output_id(branch_id)
    
        # sample from child
        sample(module.children[child_id], data, ll_cache=ll_cache, sampling_context=SamplingContext(child_sample_ids, [[output_id] for _ in range(len(child_sample_ids))]))
        

    return data

##### Examples

In [24]:
l1 = LeafNode(Scope(SPN(), [0])) # leaf node with (scope 0)
l2 = LeafNode(Scope(SPN(), [0])) # leaf node with (scope 1)
s = SumNode(children=[l1,l2]) # sum node over both product nodes (scope 0,1)

samples = sample(s)
print(f"Sample: {samples}, Sample likelihood: {likelihood(s, samples)}")

Sample: [[1.03918384]], Sample likelihood: [[0.23249419]]


In [25]:
l1 = LeafNode(Scope(SPN(), [0])) # leaf node with (scope 0)
l2 = LeafNode(Scope(SPN(), [1])) # leaf node with (scope 1)
p1 = ProductNode([l1,l2]) # product node over both leaf nodes (scope 0,1)
p2 = ProductNode([l1,l2]) # product node over both leaf nodes (scope 0,1)
s = SumNode(children=[p1,p2]) # sum node over both product nodes (scope 0,1)

data = np.random.randn(3,2)
#data[0][0] = np.nan
print(data)

samples = sample(s, data)
print(f"Sample: {samples}, Sample likelihood: {likelihood(s, samples)}")
print(f"Scope: {scope(s)}")

[[ 0.406048    1.94702653]
 [ 0.67333241 -0.29821458]
 [-1.81747117 -0.23109304]]
Sample: [[ 0.406048    1.94702653]
 [ 0.67333241 -0.29821458]
 [-1.81747117 -0.23109304]], Sample likelihood: [[0.02202074]
 [0.12135547]
 [0.02971273]]
Scope: [[Scope({0, 1})]]


In [26]:
data = np.random.randn(1,2)

# call without passing cache
l = likelihood(s, data)
print(f"Likelihood (w/o cache):\t\t{l}")

# use cache this time
d = {}
l = likelihood(s, data, cache=d)
print(f"Likelihood (store cache):\t{l}, matches stored value: {all(l == d[s])}")

# set cache to check if stored value is used
d[s] = np.ones((1,1))
l = likelihood(s, data, cache=d)
print(f"Likelihood (modified cache):\t{l}")

Likelihood (w/o cache):		[[0.15453114]]
Likelihood (store cache):	[[0.15453114]], matches stored value: True
Likelihood (modified cache):	[[1.]]


In [27]:
l = LeafNode(Scope(SPN(), [0]))

d = sample(l)
print("Without any specifications:", d.shape)

d = sample(l, 5)
print("Specifying number of samples:", d.shape)

d = np.full((3,1), np.nan)
d_ = sample(l, d)
print("Passing data tensor:", all(d == d_))

Without any specifications: (1, 1)
Specifying number of samples: (5, 1)
Passing data tensor: True


## Nested Modules

We'd like to build more complex modules from basic nodes. These could then again be combined to create even more intricate modules. For that we need to be able to nest modules.

A network without any open non-terminal nodes/modules, can straight-forwardly be nested.

However, non-terminal modules need child modules to be specified at creation. In nested modules this would require internal non-terminal modules to reference the same child modules as the enclosing module. This would be extremely messy.

Instead, one could use placeholder modules that can stand-in for the actual children. The enclosing module can the set the cache for `scope`,`likelihood` calls to these modules or redirect `sample`-calls to the actual child modules. This also allows to divide up inputs from child modules for the internal/nested modules and change their order (in contrast to direct parent-child relationships).

![nesting_module](uml/nesting_module.svg)

In [28]:
class NestingModule(Module, ABC):
    """Convenient module class for nesting non-terminal modules.
    
    Args:
        childen: list of child modules.
    """
    def __init__(self, children: List[Module]=[]) -> None:
        super(NestingModule, self).__init__(children)
        self.placeholders = []

    def create_placeholder(self, input_ids: List[int]) -> "Placeholder":
        """Creates a placholder module that can be used for internal non-terminal modules.
        
        Also registers the placeholder internally.
        """
        # create and register placeholder
        ph = self.Placeholder(self, input_ids)
        self.placeholders.append(ph)

        return ph
    
    def set_placeholders(self, cache, inputs) -> None:
        """Fills the cache for all registered placeholder modules given specified input values."""
        for ph in self.placeholders:
            # fill placeholder cache with specified input values
            cache[ph] = inputs[:,ph.input_ids]

    class Placeholder(Module):
        """Placeholder module as an intermediary module between nested non-terminal modules and actual child modules."""
        def __init__(self, owner: Module, input_ids: List[int]) -> None:
            self.owner = owner
            self.input_ids = input_ids
            
            # compute conversion from input ids corresponding child and output id (Saves computation at run-time)
            self.input_to_output_id_dict = {}
            
            #print("input ids:", self.input_ids)

            for input_id in range(len(input_ids)):
                # convert placeholder input id to actual input id
                #print(input_id)
                input_id_actual = self.input_ids[input_id]

                # set corresponding child and output id via owner
                self.input_to_output_id_dict[input_id] = self.owner.input_to_output_id(input_id_actual)

        @property
        def n_out(self) -> int:
            return len(self.input_ids)

In [29]:
@dispatch(NestingModule.Placeholder, np.ndarray, cache=dict)
@memoize
def likelihood(module: NestingModule.Placeholder, data: np.ndarray, cache: dict={}):
    """Gets called if values for placeholder module are not in the cache. In that case raise an error."""
    raise LookupError("Likelihood values for placeholder module not found in cache. Check if these are correctly set by the nesting module.")

@dispatch(NestingModule.Placeholder, cache=dict)
@memoize
def scope(module: NestingModule.Placeholder, cache: dict={}):
    """Gets called if values for placeholder module are not in the cache. In that case raise an error."""
    raise LookupError("Scope values for placeholder module not found in cache. Check if these are correctly set by the nesting module.")

@dispatch(NestingModule.Placeholder, np.ndarray, ll_cache=dict, sampling_context=SamplingContext)
def sample(module: NestingModule.Placeholder, data: np.ndarray, ll_cache: dict={}, sampling_context=None):
    """Redirects sampling calls to sample actual child modules"""
    
    if sampling_context:
        if not sampling_context.is_valid(data, module):
            # invalid sampling context
            raise ValueError(f"Specified sampling context is invalid for specified data array and module.")
    else:
        if(module.n_out != 1):
            raise ValueError("No sampling context specified. It is unclear which output to sample from.")
        else:
            sampling_context = SamplingContext(list(range(len(data.shape[0]))), [[] for _ in len(data.shape[0])])

    sampling_context_per_child = {}

    # TODO: could potentially be done more efficiently via grouping
    for instance_id, instance_output_ids in zip(sampling_context.instance_ids, sampling_context.output_ids):
       
        output_per_child = {}
        
        # iterate over actual child and output ids
        if instance_output_ids == []:
            # all children
        
            for _, ids in module.input_to_output_id_dict.items():
                output_per_child[ids[0]] = [ids[1]]
        else:
            for child_id, output_id in [module.input_to_output_id(output_id) for output_id in instance_output_ids]:

                # sort output ids per child id
                if(child_id in output_per_child):
                    output_per_child[child_id].append(output_id)
                else:
                    output_per_child[child_id] = [output_id]
        
        # append (or create) sampling contexts
        for child_id, output_ids in output_per_child.items():
            if(child_id) in sampling_context_per_child:
                sampling_context_per_child[child_id].instance_ids.append(instance_id)
                sampling_context_per_child[child_id].output_ids.append(output_ids)
            else:
                sampling_context_per_child[child_id] = SamplingContext([instance_id], [output_ids])
   
    # sample from children
    for child_id, child_sampling_context in sampling_context_per_child.items():
        sample(module.owner.children[child_id], data, ll_cache=ll_cache, sampling_context=child_sampling_context)
        

### Layer Modules

Basic `LeafLayer`,`SumLayer` and `ProducLayer` classes.

![layers](uml/layers.svg)

#### Leaf Layer

Note: `LeafLayer` only contains terminal modules and therefore has no need for placeholders.

In [39]:
class LeafLayer(Module):
    """Layer of multiple leaf nodes over the same scope.
    
    Args:
        scope: scope of all leaf nodes.
        n_out: number of leaf nodes.
    """
    def __init__(self, scope: Scope, n_out) -> None:
        
        super(LeafLayer, self).__init__()
        
        self.nodes = []
        self.scope = scope

        # create leaf nodes
        for _ in range(n_out):
            self.nodes.append(LeafNode(scope=scope))
    
    @property
    def n_out(self) -> int:
        return len(self.nodes)

@dispatch(LeafLayer, cache=dict)
@memoize
def scope(module: LeafLayer, cache: dict={}) -> ScopeArray:
    """Concatenates the scopes of all leaf nodes."""
    return np.concatenate([scope(node, cache=cache) for node in module.nodes], axis=1)

@dispatch(LeafLayer, np.ndarray, cache=dict)
@memoize
def likelihood(module: LeafLayer, data: np.ndarray, cache: dict={}) -> np.ndarray:
    """Concatenates the likelihoods for all leaf nodes."""
    return np.concatenate([likelihood(node, data, cache=cache) for node in module.nodes], axis=1)

@dispatch(LeafLayer, np.ndarray, ll_cache=dict, sampling_context=SamplingContext)
def sample(module: LeafLayer, data: np.ndarray, ll_cache: dict={}, sampling_context=None) -> np.ndarray:
    """Samples leaf nodes accoding to sampling context."""
    #print(sampling_context.instance_ids)

    if sampling_context:
        if not sampling_context.is_valid(data, module):
            # invalid sampling context
            raise ValueError(f"Specified sampling context is invalid for specified data array and module.")
    else:
        raise ValueError("No sampling context specified. It is unclear which output to sample from.")
        # create sampling context (assume all instances and all output nodes are to be used)
        #sampling_context = SamplingContext(list(range(data.shape[0])), [[] for _ in range(data.shape[0])])
    
    for node_ids in np.unique(sampling_context.output_ids, axis=0):
        if(len(node_ids) != 1):
            raise ValueError("Too many output ids specified for outputs over same scope.")
        
        node_id = node_ids[0]
        node_instance_ids = np.where(sampling_context.output_ids == node_ids)[0]
        node_instance_ids_test = sampling_context.instance_ids
        
        sample(module.nodes[node_id], data, ll_cache=ll_cache, sampling_context=SamplingContext(node_instance_ids_test, [[] for i in node_instance_ids]))
    return data

##### Examples

In [40]:
layer = LeafLayer(Scope(SPN(), [0]), 3)

data = np.random.randn(2,1)
print(f"Likelihood: {likelihood(layer, data).shape}")

print(f"Scope: {scope(layer)}")

Likelihood: (2, 3)
Scope: [[Scope({0}) Scope({0}) Scope({0})]]


In [41]:
s = SumNode(children=[layer])
sample(s)

array([[-0.08297686]])

#### Sum Layer

Because this layer now uses non-terminal modules/nodes, we use `NestingModule` as a blueprint.

In [42]:
class SumLayer(NestingModule):
    """Layer of multiple sum nodes over the same child modules.
    
    Args:
        n_out: number of sum nodes.
        children: list of child modules.
    """
    def __init__(self, n_out, children) -> None:
        super(SumLayer, self).__init__(children=children)
        
        self.nodes = []

        # all nodes share the same input link here
        ph = self.create_placeholder(list(range(self.n_in)))

        for _ in range(n_out):
            self.nodes.append(SumNode([ph]))

    @property
    def n_out(self) -> int:
        return len(self.nodes)

@dispatch(SumLayer, cache=dict)
@memoize
def scope(module: SumLayer, cache: dict={}) -> ScopeArray:
    """Concatenates the scopes of all sum nodes."""
    input_scopes = np.concatenate([scope(child, cache=cache) for child in module.children], axis=1)

    # set placeholders
    module.set_placeholders(cache, input_scopes)
    
    # compute output scopes
    output_scopes = np.concatenate([scope(node, cache=cache) for node in module.nodes], axis=1)
    
    return output_scopes

@dispatch(SumLayer, np.ndarray, cache=dict)
@memoize
def likelihood(module: SumLayer, data: np.ndarray, cache: dict={}) -> np.ndarray:
    """Concatenates the likelihoods for all sum nodes."""
    input_likelihoods = np.concatenate([likelihood(child, data, cache=cache) for child in module.children], axis=1)
    
    # set placeholders
    module.set_placeholders(cache, input_likelihoods)
    
    # compute output likelihoods
    output_scopes = np.concatenate([likelihood(node, data, cache=cache) for node in module.nodes], axis=1)
    
    return output_scopes

@dispatch(SumLayer, np.ndarray, ll_cache=dict, sampling_context=SamplingContext)
def sample(module: SumLayer, data: np.ndarray, ll_cache: dict={}, sampling_context=None) -> np.ndarray:
    """Samples leaf nodes accoding to sampling context."""
    
    if sampling_context:
        if not sampling_context.is_valid(data, module):
            # invalid sampling context
            raise ValueError(f"Specified sampling context is invalid for specified data array and module.")
    else:
        raise ValueError("No sampling context specified. It is unclear which output to sample from.")

    #print("sum layer sample")
    #print(sampling_context.instance_ids)
    #print(sampling_context.output_ids)
    # fix for [] case
    for node_ids in np.unique(sampling_context.output_ids, axis=0):
        if(len(node_ids) != 1):
            raise ValueError("Too many output ids specified for outputs over same scope.")

        node_id = node_ids[0]
        node_instance_ids = np.where(sampling_context.output_ids == node_ids)[0]

        sample(module.nodes[node_id], data, ll_cache=ll_cache, sampling_context=SamplingContext(node_instance_ids, [[] for i in node_instance_ids]))

    return data

##### Example

In [43]:
leaf_layer = LeafLayer(Scope(SPN(), [0]), 3)
sum_layer = SumLayer(3, [leaf_layer])

data = np.random.randn(1,1)
print(f"Likelihood: {likelihood(sum_layer, data)}")

print(f"Scope: {scope(sum_layer)}")


Likelihood: [[0.37929153 0.37929153 0.37929153]]
Scope: [[Scope({0}) Scope({0}) Scope({0})]]


In [44]:
s = SumNode([leaf_layer])
s1 = SumNode([LeafNode(Scope(SPN(), [0])), LeafNode(Scope(SPN(), [0])), LeafNode(Scope(SPN(), [0]))])

#print(sample(s))
#print(sample(s1))
print(f"Sample1: {np.mean(sample(s, 100000), axis=0)}")
print(f"Sample2: {np.mean(sample(s1, 100000), axis=0)}")


Sample1: [-0.00090164]
Sample2: [-0.00423772]


#### ProductLayer Class

Number of `ProductNode`s is the number of combinations of elements from each input group

In [45]:
class ProductLayer(NestingModule):
    """Layer of multiple product nodes over the same child modules.
    
    Creates a product node for each combination of inputs from the child modules.
    E.g. for two modules with 2 (ids 0,1) and 3 (ids 2,3,4) outputs, respectively, one gets nodes with the following inputs:
        [0,2]
        [0,3]
        [0,4]
        [1,2]
        [1,3]
        [1,4]

    Args:
        children: list of child modules.
    """
    def __init__(self, children) -> None:
        super(ProductLayer, self).__init__(children)
        
        self.nodes = []
        self.input_placeholders = []
        
        children_n_out = [child.n_out for child in self.children]
        total_ids = list(range(sum(children_n_out)))
        factorized_ids = []
        
        for n in children_n_out:
            factorized_ids.append(total_ids[:n])
            total_ids = total_ids[n:]

        self.input_ids_per_node = list(itertools.product(*factorized_ids))
        
        # create product nodes
        for ids in self.input_ids_per_node:
            ph = self.create_placeholder(list(ids))
            self.nodes.append(ProductNode(children=[ph]))
    
    @property
    def n_out(self) -> int:
        return len(self.nodes)

@dispatch(ProductLayer, cache=dict)
@memoize
def scope(module: ProductLayer, cache: dict={}) -> ScopeArray:
    """Concatenates the scopes of all sum nodes."""
    input_scopes = np.concatenate([scope(child, cache=cache) for child in module.children], axis=1)

    # set placeholders
    module.set_placeholders(cache, input_scopes)
    
    # compute output scopes
    output_scopes = np.concatenate([scope(node, cache=cache) for node in module.nodes], axis=1)
    
    return output_scopes

@dispatch(ProductLayer, np.ndarray, cache=dict)
@memoize
def likelihood(module: ProductLayer, data: np.ndarray, cache: dict={}) -> np.ndarray:
    """Concatenates the likelihoods for all sum nodes."""
    input_likelihoods = np.concatenate([likelihood(child, data, cache=cache) for child in module.children], axis=1)
    
    # set placeholders
    module.set_placeholders(cache, input_likelihoods)
    
    # compute output likelihoods
    output_scopes = np.concatenate([likelihood(node, data, cache=cache) for node in module.nodes], axis=1)
    
    return output_scopes

@dispatch(ProductLayer, np.ndarray, ll_cache=dict, sampling_context=SamplingContext)
def sample(module: ProductLayer, data: np.ndarray, ll_cache: dict={}, sampling_context=None) -> np.ndarray:
    """Samples leaf nodes accoding to sampling context."""
    
    if sampling_context:
        if not sampling_context.is_valid(data, module):
            # invalid sampling context
            raise ValueError(f"Specified sampling context is invalid for specified data array and module.")
    else:
        raise ValueError("No sampling context specified. It is unclear which output to sample from.")


    for node_ids in np.unique(sampling_context.output_ids, axis=0):
        if(len(node_ids) != 1):
            raise ValueError("Too many output ids specified for outputs over same scope.")

        node_id = node_ids[0]
        node_instance_ids = np.where(sampling_context.output_ids == node_ids)[0]
        sample(module.nodes[node_id], data, ll_cache=ll_cache, sampling_context=SamplingContext(node_instance_ids, [[] for i in node_instance_ids]))
        

    return data

##### Examples

In [46]:
leaf_layer_1 = LeafLayer(Scope(SPN(), [0]), 2)
leaf_layer_2 = LeafLayer(Scope(SPN(), [1]), 2)
product_layer = ProductLayer([leaf_layer_1, leaf_layer_2])

data = np.random.randn(1,2)
print(f"Likelihood: {likelihood(product_layer, data)}")

print(f"Scope: {scope(product_layer)}")

Likelihood: [[0.07119906 0.07119906 0.07119906 0.07119906]]
Scope: [[Scope({0, 1}) Scope({0, 1}) Scope({0, 1}) Scope({0, 1})]]


In [47]:
l1 = LeafLayer(Scope(SPN(), [0]), n_out=3)
s1 = SumLayer(3, [l1])

l2 = LeafLayer(Scope(SPN(), [1]), n_out=3)
s2 = SumLayer(3, [l2])

p = ProductLayer([s1,s2])
s = SumNode([p])

data = np.random.randn(1,2)
print(f"Likelihood: {likelihood(s, data)}")

print(f"Scope: {scope(s)}")

Likelihood: [[0.07730176]]
Scope: [[Scope({0, 1})]]


In [48]:
sample(s)

array([[1.45662226, 0.02762445]])

In [49]:
l1 = LeafLayer(Scope(SPN(), [0]), n_out=2)
s1 = SumLayer(2, [l1])

l2 = LeafLayer(Scope(SPN(), [1]), n_out=2)
s2 = SumLayer(2, [l2])

p = ProductLayer([s1,s2])
s = SumNode([p])

lf1 = LeafNode(Scope(SPN(), [0]))
lf2 = LeafNode(Scope(SPN(), [0]))
lf3 = LeafNode(Scope(SPN(), [1]))
lf4 = LeafNode(Scope(SPN(), [1]))

s11 = SumNode([lf1, lf2])
s12 = SumNode([lf1, lf2])
s21 = SumNode([lf3, lf4])
s22 = SumNode([lf3, lf4])

p1 = ProductNode([s11, s21])
p2 = ProductNode([s11, s22])
p3 = ProductNode([s12, s21])
p4 = ProductNode([s12, s22])

s1 = SumNode([p1, p2, p3, p4])

data = np.random.randn(1,2)
print(f"Likelihood: {likelihood(s, data)}")
print(f"Likelihood2: {likelihood(s1, data)}")


print(f"Scope: {scope(s)}")
print(f"Scope2: {scope(s1)}")
print(f"Sample1: {np.mean(sample(s, 100000), axis=0)}")
print(f"Sample2: {np.mean(sample(s1, 100000), axis=0)}")

Likelihood: [[0.05911936]]
Likelihood2: [[0.05911936]]
Scope: [[Scope({0, 1})]]
Scope2: [[Scope({0, 1})]]
Sample1: [-0.00088852 -0.00579437]
Sample2: [0.00094516 0.00203483]


## Networks

We could now build even more complex modules or networks

In [82]:
class ExampleNetwork(NestingModule):
    """Example network using layers and nodes.
    
    Args:
        children: list of child modules.
    """
    def __init__(self, children: List[Module]) -> None:
        super(ExampleNetwork, self).__init__(children)
        
        n_ins = [child.n_out for child in self.children]
        placeholders = []
        
        total_ids = range(sum(n_ins))

        for n in n_ins:
            placeholders.append(self.create_placeholder(total_ids[:n]))
            total_ids = total_ids[n:]
        
        # create product layer on top
        self.product_layer = ProductLayer(children=placeholders)
        
        # sum over all product layers
        self.sum_node = SumNode(children=[self.product_layer])
    
    @property
    def n_out(self) -> int:
        return 1

In this case we can dispatch to the sum node

In [695]:
@dispatch(ExampleNetwork, cache=dict)
@memoize
def scope(module: ExampleNetwork, cache: dict={}) -> ScopeArray:
    # compute input scopes
    input_scopes = np.concatenate([scope(child, cache=cache) for child in module.children], axis=1)
    
    # set placeholders
    module.set_placeholders(cache, input_scopes)
    
    return scope(module.sum_node, cache=cache)

@dispatch(ExampleNetwork, np.ndarray, cache=dict)
@memoize
def likelihood(module: ExampleNetwork, data: np.ndarray, cache: dict={}) -> np.ndarray:
    # compute input likelihoods
    input_likelihoods = np.concatenate([likelihood(child, data, cache=cache) for child in module.children], axis=1)
    
    # set placeholders
    module.set_placeholders(cache, input_likelihoods)
    
    return likelihood(module.sum_node, data, cache=cache)

@dispatch(ExampleNetwork, np.ndarray, ll_cache=dict, sampling_context=SamplingContext)
def sample(module: ExampleNetwork, data: np.ndarray, ll_cache: dict={}, sampling_context=None) -> np.ndarray:
    likelihood(module, data, cache=ll_cache)
    return sample(module.sum_node, data, ll_cache=ll_cache, sampling_context=sampling_context)

##### Example

In [696]:
leaf_layers = [LeafLayer(Scope(SPN(), [i]), 2) for i in range(3)]
net = ExampleNetwork(children=leaf_layers)

data = np.random.rand(2,3)

print(f"Likelihoods: {likelihood(net, data)}")
print(f"Scope: {scope(net)}")

placeholder test
range(0, 2)
{0: (0, 0), 1: (0, 1), 2: (1, 0), 3: (1, 1), 4: (2, 0), 5: (2, 1)}
input test
0
0
1
1
placeholder test
range(2, 4)
{0: (0, 0), 1: (0, 1), 2: (1, 0), 3: (1, 1), 4: (2, 0), 5: (2, 1)}
input test
0
2
1
3
placeholder test
range(4, 6)
{0: (0, 0), 1: (0, 1), 2: (1, 0), 3: (1, 1), 4: (2, 0), 5: (2, 1)}
input test
0
4
1
5
[0, 1, 2, 3, 4, 5]
[2, 3, 4, 5]
[4, 5]
[]
init prod layer
[2, 2, 2]
[]
[[0, 1], [2, 3], [4, 5]]
[(0, 2, 4), (0, 2, 5), (0, 3, 4), (0, 3, 5), (1, 2, 4), (1, 2, 5), (1, 3, 4), (1, 3, 5)]
prod_ids test
[0, 2, 4]
placeholder test
[0, 2, 4]
{0: (0, 0), 1: (0, 1), 2: (1, 0), 3: (1, 1), 4: (2, 0), 5: (2, 1)}
input test
0
0
1
2
2
4
prod_ids test
[0, 2, 5]
placeholder test
[0, 2, 5]
{0: (0, 0), 1: (0, 1), 2: (1, 0), 3: (1, 1), 4: (2, 0), 5: (2, 1)}
input test
0
0
1
2
2
5
prod_ids test
[0, 3, 4]
placeholder test
[0, 3, 4]
{0: (0, 0), 1: (0, 1), 2: (1, 0), 3: (1, 1), 4: (2, 0), 5: (2, 1)}
input test
0
0
1
3
2
4
prod_ids test
[0, 3, 5]
placeholder test
[0, 3,

## TODOs/Open Questions

* There is still some bug in `ProductLayer` that returns `NaN`s when sampling (not a concept issue, but should be fixed for better demonstration)
* Examples for optimized layers without explicit nodes.
* Structural marginalization
* Must make sure that likelihoods are computed in the very beginning and don't change (i.e. partly sampled data should not affect the likelihoods of the rest of the sampling routine)
* Creating default sampling contexts, computing input (scopes, likelihoods) etc. could be handled via specific decorator for sampling, scope and likelihood computation
* Sampling multiple non-disjoint outputs at the same time (e.g. different replicas in RAT-SPNs)
* Case distinction for scopes (e.g. merging based on sum-node different than product node). So far, modules themselves would have to check scopes manually.
* Scope variable order: e.g. for `LeafNode` the order of the scope might be important, but is a `set` here. For example, a multivariate Gaussian could be specified via scope `[0,1]` or `[1,0]`, depending on the desired order. Since the data is selected via the `set`, the order might be different from the specified `mean`,`std` values, which might not be what the user expects.
* Incorporate conditional modules (C-SPNs) and therefore evidence variables in scopes.

## Conditional Modules

One might want to create conditional modules, e.g. modules where parameters are conditioned on some inputs and set accordingly.

In [697]:
class ConditionalModule(Module):
    pass